In [1]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from sentence_transformers.evaluation import BinaryClassificationEvaluator
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import wandb
import os
wandb.init(project="product-matching-finetune", name="mpnet-finetune")


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\naeko\_netrc
wandb: Currently logged in as: naekorashi (naekorashi-smkn-3-bandung) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
df = [
    ("nivea pearl beauty 150ml", "nivea micellair pearl bright 125ml", 0),
    ("lifebuoy sabun cair 450ml", "lifebuoy body wash 450 ml", 1),
]
df = pd.DataFrame(df,columns=["ocr_product", "pusat_product", "label"])

In [7]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['label'])

train_examples = [InputExample(texts=[row['ocr_product'], row['pusat_product']], label=float(row['label'])) for idx, row in train_df.iterrows()]
test_examples = [InputExample(texts=[row['ocr_product'], row['pusat_product']], label=float(row['label'])) for idx, row in test_df.iterrows()]


KeyError: 'label'

In [ ]:
model = SentenceTransformer('paraphrase-mpnet-base-v2')

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)


In [ ]:
sentences1 = [example.texts[0] for example in test_examples]
sentences2 = [example.texts[1] for example in test_examples]
labels = [example.label for example in test_examples]

evaluator = BinaryClassificationEvaluator(sentences1, sentences2, labels)


In [ ]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=3,
    warmup_steps=100,
    output_path='./fine-tuned-product-mpnet'
)


In [ ]:
embeddings1 = model.encode(sentences1)
embeddings2 = model.encode(sentences2)

from sentence_transformers.util import cos_sim
predictions = [cos_sim(e1, e2).item() for e1, e2 in zip(embeddings1, embeddings2)]


In [ ]:
threshold = 0.6
predicted_labels = [1 if p >= threshold else 0 for p in predictions]


In [ ]:
accuracy = accuracy_score(labels, predicted_labels)
precision = precision_score(labels, predicted_labels)
recall = recall_score(labels, predicted_labels)
f1 = f1_score(labels, predicted_labels)
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


In [ ]:
wandb.log({
    "accuracy": accuracy,
    "precision": precision,
    "recall": recall,
    "f1_score": f1
})

wandb.finish()
